#Setting

In [1]:
!pip install transformers==4.17 datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [2]:
from datasets import load_dataset

dataset = load_dataset('klue', 'ner')
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/21008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 21008
    })
    validation: Dataset({
        features: ['sentence', 'tokens', 'ner_tags'],
        num_rows: 5000
    })
})

# Fine-tune a pretrained model

## Prepare a dataset

In [3]:
ner_feature = dataset["train"].features['ner_tags']
ner_feature

Sequence(feature=ClassLabel(names=['B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O'], id=None), length=-1, id=None)

In [4]:
# 사람(PS), 위치(LC), 기관(OG), 날짜(DT), 시간(TI), 수량(QT), O: 아무것도 아님

In [5]:
label_names = ner_feature.feature.names
label_names

['B-DT',
 'I-DT',
 'B-LC',
 'I-LC',
 'B-OG',
 'I-OG',
 'B-PS',
 'I-PS',
 'B-QT',
 'I-QT',
 'B-TI',
 'I-TI',
 'O']

In [6]:
print(dataset['train'][0]['sentence'])
print(dataset['train'][0]['tokens'])
print(dataset['train'][0]['ner_tags'])

특히 <영동고속도로:LC> <강릉:LC> 방향 <문막휴게소:LC>에서 <만종분기점:LC>까지 <5㎞:QT> 구간에는 승용차 전용 임시 갓길차로제를 운영하기로 했다.
['특', '히', ' ', '영', '동', '고', '속', '도', '로', ' ', '강', '릉', ' ', '방', '향', ' ', '문', '막', '휴', '게', '소', '에', '서', ' ', '만', '종', '분', '기', '점', '까', '지', ' ', '5', '㎞', ' ', '구', '간', '에', '는', ' ', '승', '용', '차', ' ', '전', '용', ' ', '임', '시', ' ', '갓', '길', '차', '로', '제', '를', ' ', '운', '영', '하', '기', '로', ' ', '했', '다', '.']
[12, 12, 12, 2, 3, 3, 3, 3, 3, 12, 2, 3, 12, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]


As usual, our texts need to be converted to token IDs before the model can make sense of them. As we saw in Chapter 6, a big difference in the case of token classification tasks is that we have pre-tokenized inputs. Fortunately, the tokenizer API can deal with that pretty easily; we just need to warn the tokenizer with a special flag.  To begin, let’s create our tokenizer object. As we said before, we will be using a BERT pretrained model, so we’ll start by downloading and caching the associated tokenizer:

평소와 같이 모델이 텍스트를 이해하려면 먼저 텍스트를 토큰 ID로 변환해야 합니다. 6장에서 보았듯이 토큰 분류 작업의 경우 큰 차이점은 사전 토큰화된 입력이 있다는 것입니다. 다행스럽게도 토크나이저 API는 이를 매우 쉽게 처리할 수 있습니다. 특별한 플래그를 사용하여 토크나이저에 경고하면 됩니다. 시작하려면 토크나이저 객체를 생성해 보겠습니다. 이전에 말했듯이 BERT 사전 학습 모델을 사용할 것이므로 관련 토크나이저를 다운로드하고 캐싱하는 것부터 시작하겠습니다.

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:
tokenizer.is_fast

True

In [9]:
#이미 tokenize 된 경우 is_split_into_words = True 로 설정
inputs = tokenizer(dataset["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['[CLS]', '특', '히', '영', '동', '고', '속', '도', '로', '강', '릉', '방', '향', '문', '막', '휴', '게', '소', '에', '서', '만', '종', '분', '기', '점', '까', '지', '5', '㎞', '구', '간', '에', '는', '승', '용', '차', '전', '용', '임', '시', '갓', '길', '차', '로', '제', '를', '운', '영', '하', '기', '로', '했', '다', '.', '[SEP]']


In [10]:
print(inputs.word_ids())

[None, 0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 32, 33, 35, 36, 37, 38, 40, 41, 42, 44, 45, 47, 48, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 63, 64, 65, None]


약간의 작업을 통해 토큰과 일치하도록 레이블 목록을 확장할 수 있습니다. 우리가 적용할 첫 번째 규칙은 특수 토큰이 -100. 이는 기본적으로 -100우리가 사용할 손실 함수(교차 엔트로피)에서 무시되는 인덱스이기 때문입니다. 그런 다음 각 토큰은 동일한 엔터티의 일부이므로 내부 단어를 시작하는 토큰과 동일한 레이블을 갖습니다. 단어 안에 있지만 시작 부분이 아닌 토큰의 경우 토큰이 엔터티를 시작하지 않기 때문에 B-를 I-로 대체

In [11]:
inputs

{'input_ids': [2, 1813, 1969, 1437, 856, 594, 1283, 848, 991, 553, 1026, 1129, 1904, 1091, 1037, 1956, 578, 1282, 1421, 1258, 1038, 1558, 1175, 645, 1540, 653, 1583, 25, 207, 615, 545, 1421, 793, 1324, 1468, 1632, 1537, 1468, 1510, 1325, 551, 647, 1632, 991, 1545, 1022, 1471, 1437, 1889, 645, 991, 1902, 809, 18, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [12]:
#[CLS], [SEP]과 같은 special token들의 원래 word_id가 None이라 label(ner_tag)가 없었음. 이 special token들의 word_id -100으로 변경. -100은 cross entropy(loss function)에서 무시되는 인덱스
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX -> B-XXX tag랑 I-tag랑 인덱스 차이가 1임! 그래서 %2 하는거
            # Sequence(feature=ClassLabel(names=['B-DT', 'I-DT', 'B-LC', 'I-LC', 'B-OG', 'I-OG', 'B-PS', 'I-PS', 'B-QT', 'I-QT', 'B-TI', 'I-TI', 'O'], id=None), length=-1, id=None)
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [13]:
labels = dataset["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[12, 12, 12, 2, 3, 3, 3, 3, 3, 12, 2, 3, 12, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 2, 3, 3, 3, 3, 12, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]
[-100, 12, 12, 2, 3, 3, 3, 3, 3, 2, 3, 12, 12, 2, 3, 3, 3, 3, 12, 12, 2, 3, 3, 3, 3, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, -100]


In [14]:
print(word_ids)

[None, 0, 1, 3, 4, 5, 6, 7, 8, 10, 11, 13, 14, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 32, 33, 35, 36, 37, 38, 40, 41, 42, 44, 45, 47, 48, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 63, 64, 65, None]


In [15]:
#이미 tokenize 된 경우 is_split_into_words = True 로 설정
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    #enumerate function: 컬렉션(리스트, 튜플, 문자열 등)을 반복할 때 현재 item과 아이템의 index를 함께 제공
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

컬럼(column) 중 하나의 개수에 불일치가 있습니다. 하나는 개수가 1,463이고 다른 하나는 길이가 1,000입니다. Dataset.map()에 대한 Documentation을 본 적이 있다면, 이 숫자(1000)는 매핑하는 함수(tokenize_and_split)에 전달된 총 샘플 개수라는 것을 기억할 수 있을겁니다. 여기에서 1,000개의 예제가 입력되어 1,463개의 새로운 자질들을 출력하므로 shape error가 발생했습니다.

문제는 크기가 다른 두 개의 서로 다른 데이터셋을 혼합하려고 한다는 것입니다. 이 오류에서 drug_dataset 열에는 1,000개의 예제가 있으나 우리가 새롭게 구성하려는 tokenized_dataset에는 그보다 많은 수의 예제(1,463)가 있습니다. Dataset 객체로서는 제대로 작동할 수가 없습니다. 따라서 이전 데이터셋의 열을 제거하거나 새로운 데이터셋과 동일한 크기로 만들어야 합니다. remove_columns 매개변수를 지정함으로써 이를 수행할 수 있습니다:

In [16]:
#크기가 다른 두 개의 서로 다른 데이터셋을 혼합하려고 한다는 것입니다. 이렇게 되면 Dataset 객체로서는 제대로 작동할 수가 없습니다. 따라서 이전 데이터셋의 열을 제거하거나 새로운 데이터셋과 동일한 크기로 만들어야 합니다.
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/21008 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [17]:
#it takes the tokenizer used to preprocess the inputs
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [18]:
#tokenize된 문장들 길이 같게 하고 학습할 때 loss function에서 무시할 수 있도록 -100으로 패딩하기 위해서 data_collator = DataCollatorForTokenClassification 사용
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,   12,   12,    2,    3,    3,    3,    3,    3,    2,    3,   12,
           12,    2,    3,    3,    3,    3,   12,   12,    2,    3,    3,    3,
            3,   12,   12,    8,    9,   12,   12,   12,   12,   12,   12,   12,
           12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,   12,
           12,   12,   12,   12,   12,   12, -100],
        [-100,    8,    9,    9,   12,   12,   12,   12,   12,   12,   12,   12,
           12,   12,   12,   12,   12,   12,   12, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100]])

As we can see, the second set of labels has been padded to the length of the first one using -100s.

In [19]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 12, 12, 2, 3, 3, 3, 3, 3, 2, 3, 12, 12, 2, 3, 3, 3, 3, 12, 12, 2, 3, 3, 3, 3, 12, 12, 8, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, -100]
[-100, 8, 9, 9, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, -100]


metric

In [20]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 795.6 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=2d38ad94f7e501ed31ad2911a2bef3b7580901dbe17588abc267558d4be1c80b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [21]:
#seqeval은 NER처럼 데이터가 편향되었을 때(이 경우에는 개체가 O인 경우가 많은 것처럼) f1 score, recall, precision 등을 보려고 할 때 쓰기 좋음
import evaluate

metric = evaluate.load("seqeval")

In [22]:
labels = dataset["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
print(labels)

['O', 'O', 'O', 'B-LC', 'I-LC', 'I-LC', 'I-LC', 'I-LC', 'I-LC', 'O', 'B-LC', 'I-LC', 'O', 'O', 'O', 'O', 'B-LC', 'I-LC', 'I-LC', 'I-LC', 'I-LC', 'O', 'O', 'O', 'B-LC', 'I-LC', 'I-LC', 'I-LC', 'I-LC', 'O', 'O', 'O', 'B-QT', 'I-QT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [23]:
#precision, recall, f1 score은 다 낮게 나오는데, accuracy는 높게 나오는 모습을 볼 수 있다
predictions = labels.copy()
predictions[3] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'LC': {'precision': 0.75, 'recall': 0.75, 'f1': 0.75, 'number': 4},
 'QT': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 0.8,
 'overall_recall': 0.8,
 'overall_f1': 0.8000000000000002,
 'overall_accuracy': 0.9848484848484849}

이 compute_metrics() 함수는 먼저 로그의 argmax를 사용하여 예측으로 변환합니다(평소와 같이 로그와 확률은 같은 순서이므로 softmax를 적용할 필요가 없습니다). 그런 다음 우리는 라벨과 예측을 모두 정수에서 문자열로 변환해야 합니다. 라벨이 -100인 모든 값을 제거한 다음 결과를 metric.compute() 메소드로 전달합니다.

In [24]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [25]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}
print(id2label, label2id)

{0: 'B-DT', 1: 'I-DT', 2: 'B-LC', 3: 'I-LC', 4: 'B-OG', 5: 'I-OG', 6: 'B-PS', 7: 'I-PS', 8: 'B-QT', 9: 'I-QT', 10: 'B-TI', 11: 'I-TI', 12: 'O'} {'B-DT': 0, 'I-DT': 1, 'B-LC': 2, 'I-LC': 3, 'B-OG': 4, 'I-OG': 5, 'B-PS': 6, 'I-PS': 7, 'B-QT': 8, 'I-QT': 9, 'B-TI': 10, 'I-TI': 11, 'O': 12}


In [26]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    "klue/bert-base",
    id2label=id2label,
    label2id=label2id,
)

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the mo

## Train

In [27]:
!pip install accelerate

In [28]:
from transformers import TrainingArguments

args = TrainingArguments(
    "klue-bert-base-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [29]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 21008
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7878


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.152700,0.159990,0.738941,0.749877,0.744369,0.949413
2,0.101000,0.147627,0.746046,0.784036,0.764569,0.954634
3,0.066400,0.152828,0.760603,0.796240,0.778014,0.956616


***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to klue-bert-base-finetuned-ner/checkpoint-2626
Configuration saved in klue-bert-base-finetuned-ner/checkpoint-2626/config.json
Model weights saved in klue-bert-base-finetuned-ner/checkpoint-2626/pytorch_model.bin
tokenizer config file saved in klue-bert-base-finetuned-ner/checkpoint-2626/tokenizer_config.json
Special tokens file saved in klue-bert-base-finetuned-ner/checkpoint-2626/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 8
Saving model checkpoint to klue-bert-base-finetuned-ner/checkpoint-5252
Configuration saved in klue-bert-base-finetuned-ner/checkpoint-5252/config.json
Model weights saved in klue-bert-base-finetuned-ner/checkpoint-5252/pytorch_model.bin
tokenizer config file saved in klue-bert-base-finetuned-ner/checkpoint-5252/tokenizer_config.json
Special tokens file saved in klue-bert-base-finetuned-ner/checkpoint-5252/special_tokens_m

TrainOutput(global_step=7878, training_loss=0.12691227582905854, metrics={'train_runtime': 1239.295, 'train_samples_per_second': 50.855, 'train_steps_per_second': 6.357, 'total_flos': 2529804225374064.0, 'train_loss': 0.12691227582905854, 'epoch': 3.0})

In [31]:
from transformers import pipeline


# Replace this with your own checkpoint
model_checkpoint = "/content/klue-bert-base-finetuned-ner/checkpoint-7878"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("손흥민 오늘 경기 벌써 멀티골 토트넘이 맨시티를 상대로 한 점 앞서갑니다.")
# print(token_classifier)

loading configuration file /content/klue-bert-base-finetuned-ner/checkpoint-7878/config.json
Model config BertConfig {
  "_name_or_path": "/content/klue-bert-base-finetuned-ner/checkpoint-7878",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "B-DT",
    "1": "I-DT",
    "2": "B-LC",
    "3": "I-LC",
    "4": "B-OG",
    "5": "I-OG",
    "6": "B-PS",
    "7": "I-PS",
    "8": "B-QT",
    "9": "I-QT",
    "10": "B-TI",
    "11": "I-TI",
    "12": "O"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DT": 0,
    "B-LC": 2,
    "B-OG": 4,
    "B-PS": 6,
    "B-QT": 8,
    "B-TI": 10,
    "I-DT": 1,
    "I-LC": 3,
    "I-OG": 5,
    "I-PS": 7,
    "I-QT": 9,
    "I-TI": 11,
    "O": 12
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  

[{'entity_group': 'PS',
  'score': 0.767125,
  'word': '손흥민',
  'start': 0,
  'end': 3},
 {'entity_group': 'OG',
  'score': 0.8271992,
  'word': '토트넘',
  'start': 17,
  'end': 20},
 {'entity_group': 'OG',
  'score': 0.94989073,
  'word': '맨시티',
  'start': 22,
  'end': 25},
 {'entity_group': 'QT',
  'score': 0.9793204,
  'word': '한 점',
  'start': 31,
  'end': 34}]